In [7]:
%pip install requests
%pip install elasticsearch
%pip install tqdm
%pip install bert-extractive-summarizer
%pip install transformers
%pip install torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install torch==1.5.0 torchvision==0.6.0 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
!nvidia-smi
torch.cuda.is_available()

use_cuda = not False and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

if torch.cuda.is_available():
    print('using GPU...')
    input_ids = torch.tensor(padded).to(device)
    attention_mask = torch.tensor(attention_mask).to(device)

    with torch.no_grad():
        last_hidden_states = model(input_ids, attention_mask=attention_mask)# .to(device)

print(time.ctime())



'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


NameError: name 'torch' is not defined

In [1]:
import gzip
import math
import numpy as np
import os
import pytest
import pickle
import random
import requests
import tarfile
import time
import timeit

import sys
import torch
from transformers import *
from summarizer import Summarizer

from collections import Counter
from collections import defaultdict
from elasticsearch import Elasticsearch
#from playsound import playsound
from tqdm.notebook import tqdm

# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# user_credential = user_secrets.get_gcloud_credential()
# user_secrets.set_tensorflow_credential(user_credential)

In [2]:

# Load model, model config and tokenizer via Transformers
pretrained_model = "distilbert-base-cased"

custom_config = AutoConfig.from_pretrained(pretrained_model)
custom_config.output_hidden_states=True
custom_tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
custom_model = AutoModel.from_pretrained(pretrained_model, config=custom_config)

model = Summarizer(custom_model=custom_model, custom_tokenizer=custom_tokenizer)

In [4]:
doc_ids_training = []

with open('../input/docidsfortraining/doc_id_for_training.txt', 'r') as file:
    doc_ids_training = file.readlines()[0].split()

print(len(doc_ids_training))

50640


In [33]:
with open('../README.MD','rb') as file:
    try:
        while True:
            print(next(file))
    except StopIteration:
        print('Done')
        


b'## Final project assignment\r\n'
b'\r\n'
b'### Team ID: Team-008\r\n'
b'Members: \r\n'
b'* Asahi Cantu \r\n'
b'* Shaon Rahman\r\n'
b'# Team-008\tTuesday 14:15-14:30\r\n'
b'URL [Description](https://github.com/kbalog/ir-course/tree/master/project)\r\n'
b'\r\n'
b'Project: [MS Marco Document re-ranking](https://microsoft.github.io/msmarco/)\r\n'
b'\r\n'
b'# Group project\r\n'
b'\r\n'
b'## Objectives\r\n'
b'\r\n'
b'The objective of project work is to apply the knowledge gained in the course, in a group setting, to a selected (open) information retrieval problem.\r\n'
b"Specifically, you'll first need to establish a reasonable baseline, then develop one or multiple advanced methods aiming to improve over the baseline. Using a standard test collection, you'll need to experimentally compare the baseline and advanced solutions.\r\n"
b'\r\n'
b'## Weekly meeting slots\r\n'
b'\r\n'
b'Each group is allocated a dedicated 15mins slot (during what would normally be lecture hours) to discuss their p

In [27]:
corpus = {}
lines = []
with open('../input/msmarcotrainingdoc/required_docs.tsv','rb') as file:
    for line in file:
        line = line.decode('UTF-8').split('\t')
        doc_id = line[0]
        lines.append(line)
        
for line in tqdm(lines):
    doc_id = line[0]
    doc_title = line[1].lower()
    doc_body = line[2].lower()
    summary = model(doc_body, max_length=250)
    corpus[doc_id] = (doc_title, summary)
        

KeyboardInterrupt: 

In [34]:
help(model)

Help on Summarizer in module summarizer.model_processors object:

class Summarizer(ModelProcessor)
 |  Summarizer(model: str = 'bert-large-uncased', custom_model: transformers.modeling_utils.PreTrainedModel = None, custom_tokenizer: transformers.tokenization_utils.PreTrainedTokenizer = None, hidden: int = -2, reduce_option: str = 'mean', sentence_handler: summarizer.sentence_handler.SentenceHandler = <summarizer.sentence_handler.SentenceHandler object at 0x0000021FA7991700>, random_state: int = 12345)
 |  
 |  Method resolution order:
 |      Summarizer
 |      ModelProcessor
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, model: str = 'bert-large-uncased', custom_model: transformers.modeling_utils.PreTrainedModel = None, custom_tokenizer: transformers.tokenization_utils.PreTrainedTokenizer = None, hidden: int = -2, reduce_option: str = 'mean', sentence_handler: summarizer.sentence_handler.SentenceHandler = <summarizer.sentence_handler.SentenceHandler obj

In [ ]:

with  gzip.GzipFile('MS-MARCO/summarized_docs.tsv.gz', 'wb') as gz_file:
    for key, val in tqdm(corpus.items()):
        title, body = val
        gz_file.write(f'{key}\t{title}\t{body}\n')